In [15]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
MODEL_NAME = "gpt-4o-mini"



In [19]:
import requests
from bs4 import BeautifulSoup

RSS_URL = "https://techcrunch.com/tag/artificial-intelligence/feed/"

res = requests.get(RSS_URL)
res.encoding = res.apparent_encoding

soup = BeautifulSoup(res.text, "xml")
items = soup.find_all("item")

rows = []
for it in items[:4]:
    title = it.title.get_text(strip=True)
    link = it.link.get_text(strip=True)
    date = it.pubDate.get_text(strip=True) if it.pubDate else ""
    desc = it.description.get_text(strip=True) if it.description else ""

    rows.append(
        f"Date: {date}\nTitle: {title}\nLink: {link}\nBody: {desc}\n--------------------"
    )

news_text = "\n".join(rows)
print(news_text)

Date: Fri, 31 Oct 2025 19:26:30 +0000
Title: Meta bought 1 GW of solar this week
Link: https://techcrunch.com/2025/10/31/meta-bought-1-gw-of-solar-this-week/
Body: The social media company inked three deals in the U.S. to power its data centers and offset its carbon footprint.
--------------------
Date: Tue, 26 Aug 2025 15:21:19 +0000
Title: How one AI startup is helping rice farmers battle climate change
Link: https://techcrunch.com/2025/08/26/how-one-ai-startup-is-helping-rice-farmers-battle-climate-change/
Body: Mitti Labs is working with The Nature Conservancy to expand the use of climate-friendly rice farming practices in India. The startup uses its AI to verify reductions in methane emissions.
--------------------
Date: Wed, 20 Aug 2025 16:00:00 +0000
Title: Harvard dropouts to launch ‘always on’ AI smart glasses that listen and record every conversation
Link: https://techcrunch.com/2025/08/20/harvard-dropouts-to-launch-always-on-ai-smart-glasses-that-listen-and-record-every-conv

In [20]:
prompt = f"""
以下は海外のAI開発ニュース一覧です。
各ニュースを日本語で、小学生にもわかるように要約してください。

# 条件：
- 出力は一覧のみ
- 各ニュースを「日付・タイトル・リンク・要約」で出力
- 要約は1件あたり80〜120文字

# 入力：
{news_text}
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=500,
    temperature=0.3
)

print(response.choices[0].message.content.strip())


- 日付: 2025年10月31日  
  タイトル: Metaが今週1GWの太陽光発電を購入  
  リンク: [こちら](https://techcrunch.com/2025/10/31/meta-bought-1-gw-of-solar-this-week/)  
  要約: Metaという会社が、データセンターを動かすためにアメリカで太陽光発電の契約を3つ結びました。これで環境に優しくなります。

- 日付: 2025年8月26日  
  タイトル: AIスタートアップが米作りを助ける  
  リンク: [こちら](https://techcrunch.com/2025/08/26/how-one-ai-startup-is-helping-rice-farmers-battle-climate-change/)  
  要約: Mitti Labsという会社が、インドの米農家を助けるためにAIを使って温室効果ガスを減らす方法を広めています。

- 日付: 2025年8月20日  
  タイトル: ハーバード中退生がAIスマートグラスを発表  
  リンク: [こちら](https://techcrunch.com/2025/08/20/harvard-dropouts-to-launch-always-on-ai-smart-glasses-that-listen-and-record-every-conversation/)  
  要約: ハーバード大学を辞めた2人が、会話をずっと聞いて記録するスマートグラスを作る会社を始めました。

- 日付: 2025年8月20日  
  タイトル: Metaが100MWの太陽光発電を追加  
  リンク: [こちら](https://techcrunch.com/2025/08/20/meta-to-add-100-mw-of-solar-power-from-u-s-gear/)  
  要約: Metaは、南カロライナ州に新しいAIデータセンターを作るために、100MWの太陽光発電を追加します。
